In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt


DATASET_DIR = "dataset"  
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
MODEL_NAME = "FaceMask.keras"


train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1)
])

train_dataset = train_dataset.map(
    lambda x, y: (augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base_model.trainable = False  

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint(MODEL_NAME, save_best_only=True),
    ReduceLROnPlateau(factor=0.2, patience=3)
]


history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

plt.figure(figsize=(12, 4))
plt.show()

print(f"Training complete! Model saved as {MODEL_NAME}")

Found 2803 files belonging to 3 classes.
Using 2243 files for training.
Found 2803 files belonging to 3 classes.
Using 560 files for validation.
Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 31s 371ms/step - accuracy: 0.3262 - loss: -0.0293 - val_accuracy: 0.3393 - val_loss: -1.1592 - learning_rate: 1.0000e-04
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 25s 341ms/step - accuracy: 0.3159 - loss: -1.2398 - val_accuracy: 0.3411 - val_loss: -2.2484 - learning_rate: 1.0000e-04
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 25s 350ms/step - accuracy: 0.3293 - loss: -2.4729 - val_accuracy: 0.3518 - val_loss: -3.6244 - learning_rate: 1.0000e-04
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 27s 372ms/step - accuracy: 0.3335 - loss: -3.6143 - val_accuracy: 0.3536 - val_loss: -5.0418 - learning_rate: 1.0000e-04
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 28s 381ms/step - accuracy: 0.3443 - loss: -4.7609 - val_accuracy: 0.3482 - val_loss: -6.5876 - learning_rate: 1.0000e-04
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 28s 393ms/step - accur

<Figure size 1200x400 with 0 Axes>

Training complete! Model saved as FaceMask.keras


In [14]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

def detect_mask():
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )
    
    model = load_model("FaceMask.keras")
    
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(60, 60)
        )

        any_no_mask = False

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            
            pred = model.predict(face, verbose=0)[0][0]
            
            label = "No Mask" if pred > 0.5 else "Mask"
            if label == "No Mask":
                any_no_mask = True
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            confidence = (1 - pred) if label == "Mask" else pred
            
            text = f"{label}: {confidence * 100:.1f}%"
            cv2.putText(frame, text, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        status_color = (0, 255, 0) if not any_no_mask else (0, 0, 255)
        status_text = "Mask Detected" if not any_no_mask else "No Mask Detected"
        
        cv2.rectangle(frame, (10, 10), (250, 50), status_color, -1)
        cv2.putText(frame, status_text, (20, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow('Face Mask Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_mask()